# Chapter 4. Spark SQL and DataFrames - Introduction to Built-in Data Sources

# Setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from pyspark import SparkContext

sc = SparkContext('local', 'Ch4')

In [3]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = (SparkSession
    .builder
    .appName("ch4 example")
    .getOrCreate())

In [4]:
spark.version

'2.4.5'

# Imports

In [6]:
import os

# Load flight data

This data is from kaggle

There is more data, and the names are different.

some of the definitions may be different too

In [10]:
data_dir = os.path.expanduser('~/dev/data/airline-kaggle')
fn_flights = os.path.join(data_dir, 'flights.csv')
fn_airlines = os.path.join(data_dir, 'airlines.csv')
fn_airports = os.path.join(data_dir, 'airports.csv')

In [11]:
fn_airlines

'/Users/bartev/dev/data/airline-kaggle/airlines.csv'

In [51]:
def lcase_cols(df):
    """return a new DataFrame with all columns lower cased"""
    return df.toDF(*[c.lower() for c in df.columns])

In [52]:
airlines = (spark
 .read
 .format('csv')
 .option('samplingRatio', 0.01)
 .option('inferSchema', True)
 .option('header', True)
 .csv(fn_airlines)
)

airlines = lcase_cols(airlines)

In [53]:
airlines.show()

+---------+--------------------+
|iata_code|             airline|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+



In [54]:
airports = (spark
 .read
 .format('csv')
 .option('inferSchema', True)
 .option('header', True)
 .csv(fn_airports)
)
airports = lcase_cols(airports)

In [21]:
airports.show()

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [96]:
flights = (spark
 .read
 .format('csv')
 .option('samplingRatio', 0.01)
 .option('inferSchema', True)
 .option('header', True)
 .csv(fn_flights)
)
flights = lcase_cols(flights)

In [50]:
flights.count()

5819079

In [97]:
# match columns with example from book
flights = (flights
          .withColumnRenamed('origin_airport', 'origin')
          .withColumnRenamed('destination_airport', 'destination')
           .withColumnRenamed('departure_delay', 'dep_delay')
           .withColumnRenamed('arrival_delay', 'delay')
          )

flights.createOrReplaceTempView('us_delay_flights_tbl')

In [98]:
flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: integer (nullable = true)
 |-- tail_number: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- scheduled_departure: integer (nullable = true)
 |-- departure_time: integer (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- taxi_out: integer (nullable = true)
 |-- wheels_off: integer (nullable = true)
 |-- scheduled_time: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- wheels_on: integer (nullable = true)
 |-- taxi_in: integer (nullable = true)
 |-- scheduled_arrival: integer (nullable = true)
 |-- arrival_time: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- diverted: in

In [77]:
flights.select('airline', 'delay', 'arrival_delay',
              'scheduled_departure', 'departure_time',
              'scheduled_time', 'elapsed_time', 'air_time').show()

+-------+-----+-------------+-------------------+--------------+--------------+------------+--------+
|airline|delay|arrival_delay|scheduled_departure|departure_time|scheduled_time|elapsed_time|air_time|
+-------+-----+-------------+-------------------+--------------+--------------+------------+--------+
|     AS|  -11|          -22|                  5|          2354|           205|         194|     169|
|     AA|   -8|           -9|                 10|             2|           280|         279|     263|
|     US|   -2|            5|                 20|            18|           286|         293|     266|
|     AA|   -5|           -9|                 20|            15|           285|         281|     258|
|     AS|   -1|          -21|                 25|            24|           235|         215|     199|
|     DL|   -5|            8|                 25|            20|           217|         230|     206|
|     NK|   -6|          -17|                 25|            19|           181|   

In [68]:
flights.show(3)

+----+-----+---+-----------+-------+-------------+-----------+------+-----------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin|destination|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|
+----+-----+---+-----------+-------+-------------+-----------+------+-----------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+----

## SQL Query-1: Find all flights whose distance between origin and destination is greater than 1000 miles in Scala or Python

In [103]:
spark.sql("""
select distance, origin, destination 
from us_delay_flights_tbl
where distance > 1000
order by distance desc
limit 10
""").show()

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
|    4983|   HNL|        JFK|
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
|    4983|   HNL|        JFK|
|    4983|   JFK|        HNL|
|    4983|   JFK|        HNL|
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
+--------+------+-----------+



In [129]:
(flights
 .select('distance', 'origin', 'destination')
 .filter('distance > 1000')
 .orderBy('distance', ascending=False)
 .show(10))

# or orderBy(desc('distance'))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
|    4983|   JFK|        HNL|
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
|    4983|   HNL|        JFK|
|    4983|   HNL|        JFK|
|    4983|   JFK|        HNL|
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [131]:
from pyspark.sql.functions import desc

(flights
 .select('distance', 'origin', 'destination')
 .filter('distance > 1000')
 .orderBy(desc('distance'))
 .show(10))

# or orderBy(desc('distance'))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
|    4983|   JFK|        HNL|
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
|    4983|   HNL|        JFK|
|    4983|   HNL|        JFK|
|    4983|   JFK|        HNL|
|    4983|   JFK|        HNL|
|    4983|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



## SQL Query-2: Find all flights with at least 2 hour delays between San Francisco (SFO) and Chicago (ORD) in Scala or Python



In [104]:
spark.sql("""
select year, month, day, delay, origin, destination
from us_delay_flights_tbl
where 1=1
and delay > 120
and origin = 'SFO'
and destination = 'ORD'
order by delay desc
""").show(10)

+----+-----+---+-----+------+-----------+
|year|month|day|delay|origin|destination|
+----+-----+---+-----+------+-----------+
|2015|   12| 13| 1187|   SFO|        ORD|
|2015|   12| 18| 1008|   SFO|        ORD|
|2015|   12| 18|  951|   SFO|        ORD|
|2015|    6| 20|  506|   SFO|        ORD|
|2015|   11| 21|  488|   SFO|        ORD|
|2015|    3| 31|  471|   SFO|        ORD|
|2015|   12| 26|  458|   SFO|        ORD|
|2015|    8| 23|  422|   SFO|        ORD|
|2015|    6| 23|  409|   SFO|        ORD|
|2015|    1|  5|  408|   SFO|        ORD|
+----+-----+---+-----+------+-----------+
only showing top 10 rows



In [132]:
(flights
 .select('year', 'month', 'day', 'delay', 'origin', 'destination')
 .filter('delay > 120')
 .filter('origin = "SFO"')
 .filter('destination = "ORD"')
 .orderBy('delay', ascending=False)
 .show(10))

+----+-----+---+-----+------+-----------+
|year|month|day|delay|origin|destination|
+----+-----+---+-----+------+-----------+
|2015|   12| 13| 1187|   SFO|        ORD|
|2015|   12| 18| 1008|   SFO|        ORD|
|2015|   12| 18|  951|   SFO|        ORD|
|2015|    6| 20|  506|   SFO|        ORD|
|2015|   11| 21|  488|   SFO|        ORD|
|2015|    3| 31|  471|   SFO|        ORD|
|2015|   12| 26|  458|   SFO|        ORD|
|2015|    8| 23|  422|   SFO|        ORD|
|2015|    6| 23|  409|   SFO|        ORD|
|2015|    1|  5|  408|   SFO|        ORD|
+----+-----+---+-----+------+-----------+
only showing top 10 rows



In [101]:
spark.sql("""
select year, count(*) as cnt
from us_delay_flights_tbl
group by year
order by cnt
""").show(30)

+----+-------+
|year|    cnt|
+----+-------+
|2015|5819079|
+----+-------+



In [134]:
flights.groupBy('year').count().show()

+----+-------+
|year|  count|
+----+-------+
|2015|5819079|
+----+-------+



In [137]:
flights.groupBy('month').count().orderBy('month').show()

+-----+------+
|month| count|
+-----+------+
|    1|469968|
|    2|429191|
|    3|504312|
|    4|485151|
|    5|496993|
|    6|503897|
|    7|520718|
|    8|510536|
|    9|464946|
|   10|486165|
|   11|467972|
|   12|479230|
+-----+------+



## SQL Query-3: A more complicated query in SQL: let’s label all US flights with a human readable label: Very Long Delay (> 6 hours), Lon g Delay (2 - 6 hours), etc. in a new column called “Flight_Delays” in Scala or Python

In [114]:
spark.sql("""
select delay,
    origin,
    destination,
    case when delay > 360 then 'very long'
         when delay > 120 then 'long'
         when delay > 60 then 'short'
         when delay > 0 then 'tolerable'
         when delay = 0 then 'no delay'
         else 'early'
    end as flight_delays
from us_delay_flights_tbl
order by delay desc
""").show(20)

+-----+------+-----------+-------------+
|delay|origin|destination|flight_delays|
+-----+------+-----------+-------------+
| 1971|   BHM|        DFW|    very long|
| 1898|   RIC|        DFW|    very long|
| 1665|   SAN|        DFW|    very long|
| 1638|   DTW|        ORD|    very long|
| 1636|   ABQ|        DFW|    very long|
| 1636|   IND|        LAX|    very long|
| 1627|   STL|        MIA|    very long|
| 1598|   OMA|        DFW|    very long|
| 1593|   LAS|        LAX|    very long|
| 1576|   HNL|        LAX|    very long|
| 1574|   HNL|        LAX|    very long|
| 1557|   MSP|        ORD|    very long|
| 1556|   MCO|        JFK|    very long|
| 1555| 14747|      11298|    very long|
| 1554|   SAT|        DFW|    very long|
| 1554|   SAN|        JFK|    very long|
| 1546|   FAT|        DFW|    very long|
| 1528| 11612|      13930|    very long|
| 1514|   SMF|        DFW|    very long|
| 1508|   RIC|        DFW|    very long|
+-----+------+-----------+-------------+
only showing top

In [142]:
from pyspark.sql.functions import col, when

In [155]:
(flights
 .select('delay', 
         'origin', 
         'destination',
         when(col('delay') > 360, 'very long')
         .when(col('delay') > 120, 'long')
         .when(col('delay') > 60, 'short')
         .when(col('delay') > 0, 'tolerable')
         .when(col('delay') == 0, 'no delay')
         .when(col('delay') < 0, 'early')
         .otherwise('unknown')
         .alias('flight_delays'))
 .orderBy(desc('delay'))
#  .groupBy('flight_delays')
#  .count()
.show())

+-----+------+-----------+-------------+
|delay|origin|destination|flight_delays|
+-----+------+-----------+-------------+
| 1971|   BHM|        DFW|    very long|
| 1898|   RIC|        DFW|    very long|
| 1665|   SAN|        DFW|    very long|
| 1638|   DTW|        ORD|    very long|
| 1636|   ABQ|        DFW|    very long|
| 1636|   IND|        LAX|    very long|
| 1627|   STL|        MIA|    very long|
| 1598|   OMA|        DFW|    very long|
| 1593|   LAS|        LAX|    very long|
| 1576|   HNL|        LAX|    very long|
| 1574|   HNL|        LAX|    very long|
| 1557|   MSP|        ORD|    very long|
| 1556|   MCO|        JFK|    very long|
| 1555| 14747|      11298|    very long|
| 1554|   SAT|        DFW|    very long|
| 1554|   SAN|        JFK|    very long|
| 1546|   FAT|        DFW|    very long|
| 1528| 11612|      13930|    very long|
| 1514|   SMF|        DFW|    very long|
| 1508|   RIC|        DFW|    very long|
+-----+------+-----------+-------------+
only showing top

In [159]:
spark

# CREATING SQL DATABASES AND TABLES

In [161]:
spark.sql("""create database learn_spark_db""")
spark.sql("use learn_spark_db")

AnalysisException: "Database 'learn_spark_db' already exists;"

## create a managed table

In [163]:
(flights
 .select('year', 'month', 'day',
         'delay', 
         'distance',
         'origin', 
         'destination')
).printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [164]:
# sql

spark.sql("""
create table managed_us_delay_flights_tbl
(year int, month int, day int, delay int, distance int,
origin string, destination string)""")

AnalysisException: "Hive support is required to CREATE Hive TABLE (AS SELECT);;\n'CreateTable `managed_us_delay_flights`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists\n"

In [165]:
# in python DataFrame API

flights.write.saveAsTable('managed_us_delay_flights_tbl')

## create an unmanaged table

In [173]:
# in python DataFrame API

data_dir = '/Users/bartev/dev/github-bv/san-tan/lrn-spark/tmp/data/us_flights_delay'
(flights.select('year', 'month', 'day', 'delay', 'distance', 'origin', 'destination')
    .write
    .option('path', data_dir)
    .saveAsTable('us_delay_flights_tbl'))

AnalysisException: 'Table `us_delay_flights_tbl` already exists.;'

## create views

In [168]:
# in python DataFrame API

In [177]:
df_sfo = spark.sql("""
select year,
    month,
    day,
    delay,
    distance,
    origin,
    destination 
from us_delay_flights_tbl 
where origin = 'SFO'""")

df_jfk = spark.sql("""
select year,
    month,
    day,
    delay,
    distance,
    origin,
    destination 
from us_delay_flights_tbl 
where origin = 'JFK'""")


In [180]:
df_sfo.createOrReplaceGlobalTempView('us_origin_airport_SFO_global_tmp_view')

df_jfk.createOrReplaceTempView('us_origin_airport_JFK_tmp_view')

Note: to query the global temp view, use the prefix `global_temp`

In [182]:
(spark
 .sql("select * from global_temp.us_origin_airport_SFO_global_tmp_view")
 .show())

+----+-----+---+-----+--------+------+-----------+
|year|month|day|delay|distance|origin|destination|
+----+-----+---+-----+--------+------+-----------+
|2015|    1|  1|    5|    2296|   SFO|        CLT|
|2015|    1|  1|    8|    1589|   SFO|        MSP|
|2015|    1|  1|  -13|    1464|   SFO|        DFW|
|2015|    1|  1|   -7|    1635|   SFO|        IAH|
|2015|    1|  1|   26|     967|   SFO|        DEN|
|2015|    1|  1|   -2|     967|   SFO|        DEN|
|2015|    1|  1|  -16|    2139|   SFO|        ATL|
|2015|    1|  1|   -4|    1846|   SFO|        ORD|
|2015|    1|  1|  -11|     337|   SFO|        LAX|
|2015|    1|  1|    3|     651|   SFO|        PHX|
|2015|    1|  1| null|    1464|   SFO|        DFW|
|2015|    1|  1|    1|    1635|   SFO|        IAH|
|2015|    1|  1|  -23|    2565|   SFO|        EWR|
|2015|    1|  1|  -14|     354|   SFO|        LGB|
|2015|    1|  1|   -2|     599|   SFO|        SLC|
|2015|    1|  1|   -2|    1635|   SFO|        IAH|
|2015|    1|  1|   -3|    2139|

In [183]:
(spark.sql("select * from us_origin_airport_JFK_tmp_view").show())

+----+-----+---+-----+--------+------+-----------+
|year|month|day|delay|distance|origin|destination|
+----+-----+---+-----+--------+------+-----------+
|2015|    1|  1|   19|    1598|   JFK|        SJU|
|2015|    1|  1|   69|    1089|   JFK|        MIA|
|2015|    1|  1|  -19|    1576|   JFK|        BQN|
|2015|    1|  1|  -22|    1028|   JFK|        PBI|
|2015|    1|  1|  -19|     944|   JFK|        MCO|
|2015|    1|  1|   -1|    1005|   JFK|        TPA|
|2015|    1|  1|  -19|     760|   JFK|        ATL|
|2015|    1|  1|   -4|    2586|   JFK|        SFO|
|2015|    1|  1|   -2|    1069|   JFK|        FLL|
|2015|    1|  1|  -23|    1598|   JFK|        SJU|
|2015|    1|  1|    5|    2153|   JFK|        PHX|
|2015|    1|  1|    4|    2475|   JFK|        LAX|
|2015|    1|  1|  -17|    1074|   JFK|        RSW|
|2015|    1|  1|    2|    1182|   JFK|        MSY|
|2015|    1|  1|   45|    2248|   JFK|        LAS|
|2015|    1|  1|  -12|     187|   JFK|        BOS|
|2015|    1|  1|   -7|    1089|

In [187]:
spark.read.table("us_origin_airport_JFK_tmp_view").show()

+----+-----+---+-----+--------+------+-----------+
|year|month|day|delay|distance|origin|destination|
+----+-----+---+-----+--------+------+-----------+
|2015|    1|  1|   19|    1598|   JFK|        SJU|
|2015|    1|  1|   69|    1089|   JFK|        MIA|
|2015|    1|  1|  -19|    1576|   JFK|        BQN|
|2015|    1|  1|  -22|    1028|   JFK|        PBI|
|2015|    1|  1|  -19|     944|   JFK|        MCO|
|2015|    1|  1|   -1|    1005|   JFK|        TPA|
|2015|    1|  1|  -19|     760|   JFK|        ATL|
|2015|    1|  1|   -4|    2586|   JFK|        SFO|
|2015|    1|  1|   -2|    1069|   JFK|        FLL|
|2015|    1|  1|  -23|    1598|   JFK|        SJU|
|2015|    1|  1|    5|    2153|   JFK|        PHX|
|2015|    1|  1|    4|    2475|   JFK|        LAX|
|2015|    1|  1|  -17|    1074|   JFK|        RSW|
|2015|    1|  1|    2|    1182|   JFK|        MSY|
|2015|    1|  1|   45|    2248|   JFK|        LAS|
|2015|    1|  1|  -12|     187|   JFK|        BOS|
|2015|    1|  1|   -7|    1089|

What is the difference between a SparkSession and a Spark application